# Парсинг НКРЯ (запрос: местоимение/сущ в им.п. + глагол)

In [ ]:
! pip install lxml
! pip install requests
! pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm 


sentences = {}

for i in tqdm(range(100)):
    url = 'https://processing.ruscorpora.ru/search.xml?gramm1=%28S%7CSPRO%29%2Cnom&gramm2=V&lang=ru&level1=0&level2=0&max2=1&min2=1&mode=main&nodia=1&p='+str(i)+'&parent1=0&parent2=0&sem-mod1=sem&sem-mod1=semx&sem-mod2=sem&sem-mod2=semx&sort=i_grtagging&text=lexgramm'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    all_sentences = soup.find_all('div', class_='content')[0].find_all('ol')[0]
    all_tbody = all_sentences.find_all('tbody')
    lis = []
    for tbody in all_tbody:
        lis.extend(tbody.find_all('li'))
    for li in lis:
        all_words = [_.text for _ in li.find_all('span')]
        for j in range(3):
            del all_words[-1]
        sent = ' '.join(all_words)
        agent_verb = [_.text for _ in li.find_all('span', class_='b-wrd-expl g-em')]
        if len(agent_verb) == 2:
            sentences[sent] = ' '.join(agent_verb)
  

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
sntnc = []
ag_vb = []

for k, v in sentences.items():
    sntnc.append(k)
    ag_vb.append(v)

ag_ind = []
vb_ind = []
for i in range(len(sntnc)):
    ag_ind.append(len(sntnc[i].split(ag_vb[i])[0].split(' '))-1)
    vb_ind.append(len(sntnc[i].split(ag_vb[i])[0].split(' ')))

sent_data = {'sentence': sntnc,
             'pair': ag_vb,
             'agent_index': ag_ind,
             'verb_index': vb_ind} 

sent_ind_df = pd.DataFrame(sent_data)

In [ ]:
sent_ind_df.to_csv('sentence_index_df.csv', sep='\t', encoding='utf-8')

In [ ]:
sent_ind_df

,sentence,pair,agent_index,verb_index
0,Например в Финляндии запретили курить везде в ...,страна обещает,13,14
1,У нас же во-первых сильное лобби со стороны та...,законы нарушаются,36,37
2,Ну что ж может на первых этапах это и будет ра...,этап имеющий,18,19
3,Поступай так чтобы твои наибольшие способности...,способности служили,5,6
4,Моя работа приносит мне не только моральное уд...,работа приносит,1,2
...,...,...,...,...
6227,Вообще же несмотря на принятую в России в 2002...,Байконур остаётся,13,14
6228,Сегодня это обходится Москве в 115 миллионов в...,это обходится,1,2
6229,В этом ключе Байконур говорит учёный не такое ...,Байконур говорит,3,4
6230,Впрочем если восстанавливать структуру космодр...,она была,9,10


# Attention

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 10.5 MB/s 
     |████████████████████████████████| 86 kB 4.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.3 MB/s 
     |████████████████████████████████| 596 kB 59.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig

model_version = 'sberbank-ai/ruBert-base'
model = BertModel.from_pretrained(model_version, output_attentions=True, num_attention_heads=12)
tokenizer = BertTokenizer.from_pretrained(model_version)


Downloading:   0%|          | 0.00/590 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

In [ ]:
from google.colab import files
import pandas as pd
import io
import re
from tqdm.auto import tqdm

uploaded = files.upload()

Saving sentence_index_df.csv to sentence_index_df.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['sentence_index_df.csv']), sep='\t')
df = df.drop(columns='Unnamed: 0')

In [ ]:
df

,sentence,pair,agent_index,verb_index
0,Например в Финляндии запретили курить везде в ...,страна обещает,13,14
1,У нас же во-первых сильное лобби со стороны та...,законы нарушаются,36,37
2,Ну что ж может на первых этапах это и будет ра...,этап имеющий,18,19
3,Поступай так чтобы твои наибольшие способности...,способности служили,5,6
4,Моя работа приносит мне не только моральное уд...,работа приносит,1,2
...,...,...,...,...
6227,Вообще же несмотря на принятую в России в 2002...,Байконур остаётся,13,14
6228,Сегодня это обходится Москве в 115 миллионов в...,это обходится,1,2
6229,В этом ключе Байконур говорит учёный не такое ...,Байконур говорит,3,4
6230,Впрочем если восстанавливать структуру космодр...,она была,9,10


In [ ]:
def gaw(sentence, attention, agent_ind, verb_ind):
  
  weights = {i:{j: None for j in range(12)} for i in range(12)}

  for l in range(len(attention)): # 12
    for k in range(len(attention[l][0])): # 2
      for i in range(len(attention[l][0][k])):
        if i == verb_ind:
          for j in range(len(attention[l][0][k][i])):
            if j == agent_ind:
              weights[l][k] = {(sentence.split(' ')[0], sentence.split(' ')[1]): float(attention[l][0][k][i][j])}

  return weights

In [ ]:
def get_max_weights(weights):
  
  a, v = list(weights[0][0][0])[0][0], list(weights[0][0][0])[0][1]
  out_weights = {i: {j: {(a, v): 0 for _ in range(12)} for j in range(12)} for i in range(12)}

  for weight_range in range(len(weights)):
    for i in range(12):
      for j in range(12):
        if out_weights[i][j][(a, v)] < weights[weight_range][i][j][(a, v)]:
          out_weights[i][j][(a, v)] = weights[weight_range][i][j][(a, v)]

  return out_weights

In [ ]:
def get_mean_weights(weights):
  
  a, v = list(weights[0][0][0])[0][0], list(weights[0][0][0])[0][1]
  out_weights = {i: {j: {(a, v): 0 for _ in range(12)} for j in range(12)} for i in range(12)}

  for weight_range in range(len(weights)):
    for i in range(12):
      for j in range(12):
        out_weights[i][j][(a, v)] += weights[weight_range][i][j][(a, v)]
  
  for i in range(12):
    for j in range(12):
      out_weights[i][j][(a, v)] = out_weights[i][j][(a, v)]/len(weights)

  return out_weights

In [ ]:
def get_all_attention_weights(sentence, attention, agent_inds, verb_inds):

  out = {'mean_bw_vtokens': None, 'max_bw_vtokens': None, 'st_bw_vtokens': None}

  if len(verb_inds) == 1 and len(agent_inds) == 1: # ЕСЛИ ВСЕ ИДЕАЛЬНО 
    for tp in out.keys():
      weights = gaw(sentence, attention, agent_inds[0], verb_inds[0])
      out[tp] = {'mean_bw_atokens':weights, 'max_bw_atokens':weights, 
                 'st_bw_atokens': weights}
  else:
    if len(verb_inds) == 1:
      ag_weights = []

      for i in range(len(agent_inds)):
        ag_weights.append(gaw(sentence, attention, agent_inds[i], verb_inds[0]))

      mean_ag_weights = get_mean_weights(ag_weights)
      max_ag_weights = get_max_weights(ag_weights)

      ag_out = {'mean_bw_atokens':mean_ag_weights, 'max_bw_atokens':max_ag_weights, 
                'st_bw_atokens': ag_weights[0]}

      for tp in out.keys():
        out[tp] = ag_out

    elif len(agent_inds) == 1:
      vb_weights = []

      for i in range(len(verb_inds)):
        vb_weights.append(gaw(sentence, attention, agent_inds[0], verb_inds[i]))

      mean_vb_weights = get_mean_weights(vb_weights)
      max_vb_weights = get_max_weights(vb_weights)

      out['mean_bw_vtokens'] = {'mean_bw_atokens':mean_vb_weights, 'max_bw_atokens':mean_vb_weights, 
                                'st_bw_atokens': mean_vb_weights}
      out['max_bw_vtokens'] = {'mean_bw_atokens':max_vb_weights, 'max_bw_atokens':max_vb_weights, 
                               'st_bw_atokens': max_vb_weights}
      out['st_bw_vtokens'] = {'mean_bw_atokens':vb_weights[0], 'max_bw_atokens':vb_weights[0], 
                              'st_bw_atokens': vb_weights[0]}

    else:
      av_weights = [] 
      for i in range(len(verb_inds)): 
        verb_ag = []
        for j in range(len(agent_inds)):
          verb_ag.append(gaw(sentence, attention, agent_inds[j], verb_inds[i]))
        av_weights.append(verb_ag)
      counted_ag_weights = []
      for av in av_weights:
        mean_av_weights = get_mean_weights(av)
        max_av_weights = get_max_weights(av)
        first_av_weights = av[0]
        counted_ag_weights.append([mean_av_weights, max_av_weights, first_av_weights])
      out['mean_bw_vtokens'] = {'mean_bw_atokens': get_mean_weights([_[0] for _ in counted_ag_weights]),
                                'max_bw_atokens': get_mean_weights([_[1] for _ in counted_ag_weights]), 
                                'st_bw_atokens': get_mean_weights([_[2] for _ in counted_ag_weights])}
      out['max_bw_vtokens'] = {'mean_bw_atokens': get_max_weights([_[0] for _ in counted_ag_weights]),
                                'max_bw_atokens': get_max_weights([_[1] for _ in counted_ag_weights]), 
                                'st_bw_atokens': get_max_weights([_[2] for _ in counted_ag_weights])}
      out['st_bw_vtokens'] = {'mean_bw_atokens': counted_ag_weights[0][0],
                                'max_bw_atokens': counted_ag_weights[0][1], 
                                'st_bw_atokens': counted_ag_weights[0][2]}

  return out            

1. mean по глаголу mean по агенсу
2. mean по глаголу max по агенсу
3. mean по глаголу first по агенсу
4. first по глаголу mean по агенсу
5. first по глаголу max по агенсу
6. first по глаголу first по агенсу
7. max по глаголу mean по агенсу
8. max по глаголу max по агенсу
9. max по глаголу first по агенсу

* если всего 1 токен глагола и 1 токен агенса, будет 9 одинаковых таблиц значений
* если 1 токен глагола и несколько агенса (или наоборот), будет 3 тройки одинаковых таблиц значений
* если все и агенс, и глагол разделились на неск токенов, будет 9 разных таблиц значений


In [ ]:
def find_indexes(tokens, sentence, agent, verb):
    remade_agent = re.sub('й', 'и', re.sub('ё', 'е', agent.lower()))
    remade_verb = re.sub('й', 'и', re.sub('ё', 'е', verb.lower()))
    sentence = sentence.lower()
    word_before_agent = sentence.split(' ')[sentence.split(' ').index(agent.lower())-1].lower()
    if word_before_agent == sentence.split(' ')[-1]:
      word_before_agent = ''
    remade_wba = re.sub('й', 'и', re.sub('ё', 'е', word_before_agent))
    sent_count = ''
    start_ind_agent, fin_ind_agent, start_ind_verb, fin_ind_verb = 0, 0, 0, 0
    for i in range(len(tokens)):
        word = re.sub('##','',tokens[i])
        word = re.sub('\[[^\]]+\]', '', word)
        if '##' in tokens[i] or '[' in tokens[i] or '-' in tokens[i]:
            sent_count += word
        else:
            if sent_count:
                if sent_count[-1]!='-':
                    sent_count += ' '+ word
                else:
                    sent_count += word
            else:
                sent_count += ' '+ word
        if sent_count.split(' ')[-1] == remade_wba:
            if start_ind_agent == 0:
                start_ind_agent = i+1
        elif sent_count.split(' ')[-1] == remade_agent:
            if fin_ind_agent == 0 or fin_ind_agent < start_ind_agent:
                fin_ind_agent = i
            if start_ind_verb == 0:
                start_ind_verb = i+1
        if sent_count.split(' ')[-1] == remade_verb:
            if fin_ind_verb == 0 or fin_ind_verb < start_ind_verb:
                fin_ind_verb = i
    agent_indexes = [i for i in range(start_ind_agent, fin_ind_agent+1)]
    verb_indexes = [i for i in range(start_ind_verb, fin_ind_verb+1)]
    return agent_indexes, verb_indexes

In [ ]:
verb_lists = ['mean_bw_vtokens', 'max_bw_vtokens', 'st_bw_vtokens']
agent_lists = ['mean_bw_atokens', 'max_bw_atokens', 'st_bw_atokens']

dicts = ['text', 'agent', 'verb', 'layer', 'head']
for d in dicts:
  locals()[d] = {vtoken:{agtoken:[] for agtoken in agent_lists} for vtoken in verb_lists}

for sentence in list(df['sentence']):
    if list(df['sentence']).index(sentence)%50 == 0:
      print(list(df['sentence']).index(sentence))
    inputs = tokenizer.encode_plus(sentence,  return_tensors='pt')
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention = model(input_ids, token_type_ids=token_type_ids)[-1]

    input_id_list = input_ids[0].tolist() 
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)

    agent_token = list(df[df['sentence']==sentence]['pair'])[0].split(' ')[0]
    verb_token = list(df[df['sentence']==sentence]['pair'])[0].split(' ')[1]
    agent_indexes, verb_indexes = find_indexes(tokens, sentence, agent_token, verb_token)
    #print(sentence, '\n', agent_token, verb_token)
    #print(agent_indexes, verb_indexes)
    
    try:
      all_types_of_weights = get_all_attention_weights(list(df[df['sentence']==sentence]['pair'])[0], attention, agent_indexes, verb_indexes)

      for v in verb_lists:
        for a in agent_lists:
          for l in list(all_types_of_weights[v][a]): # 0-12
            head_weights = []
            for h in list(all_types_of_weights[v][a][l]): # 0-12
                head_weights.append(list(all_types_of_weights[v][a][l][h].values())[0])
        
            text[v][a].append(sentence)
            agent[v][a].append(list(all_types_of_weights[v][a][0][0].keys())[0][0])
            verb[v][a].append(list(all_types_of_weights[v][a][0][0].keys())[0][1])
            layer[v][a].append(l)
            head[v][a].append(head_weights)
    except:
      pass


In [ ]:
for v in verb_lists:
  for a in agent_lists:
    for i in range(12):
      try:
        locals()['h_'+str(i)][v][a] = [h[i] for h in head[v][a]]
      except:
        try:
          locals()['h_'+str(i)][v] = {a: [h[i] for h in head[v][a]]}
        except:
          locals()['h_'+str(i)] = {v: {a: [h[i] for h in head[v][a]]}}


In [ ]:
columns = ['text', 'agent', 'verb', 'layer']
columns.extend(['h_'+str(i) for i in range(12)])

In [ ]:
for v in verb_lists:
  for a in agent_lists:
    data = {col: globals()[col][v][a] for col in columns}
    locals()['v'+v.split('_')[0]+'_a'+a.split('_')[0]] = pd.DataFrame(data)

In [ ]:
# таблички вот такие: vmean_amean, vmean_amax, vmean_ast, vmax_amean, vmax_amax,
# vmax_ast, vst_amean, vst_amax, vst_ast
# (v -- как считается между токенами глагола, a -- как между токенами агенса)

# Стандартное отклонение

In [ ]:
import statistics 


def check_if_difference_is_ok(df):

  values = {}

  ok_layers_heads = {k:{l:None for l in range(12)} for k in range(12)}

  for i in range(12): # для каждого слоя
    layer_df = df[df['layer'] == i]
    
    for j in range(12): # для каждой головы
      head_values = list(layer_df['h_'+str(j)])

      ok = 0
      not_ok = 0

      maximum = statistics.mean(head_values) + statistics.stdev(head_values)
      minimum = statistics.mean(head_values) - statistics.stdev(head_values)

      for vl in head_values:
        #print(i, j, vl, ' mean: ', statistics.mean(head_values), ' stdev: ', statistics.stdev(head_values))
        if vl < maximum and vl > minimum:
          ok += 1
        else:
          not_ok += 1
      
      if ok > not_ok:
        ok_layers_heads[i][j] = (1, 'Разница: '+str(ok-not_ok))
      else:
        ok_layers_heads[i][j] = (0, 'Разница: '+str(not_ok-ok))

  return ok_layers_heads

In [ ]:
vmax_amax[vmax_amax['layer']==0]

,text,agent,verb,layer,h_0,h_1,h_2,h_3,h_4,h_5,h_6,h_7,h_8,h_9,h_10,h_11
0,Например в Финляндии запретили курить везде в ...,страна,обещает,0,0.077476,0.064109,0.021131,0.017083,0.016226,0.010913,0.047754,0.016019,0.052807,0.665802,0.019612,0.002384
12,У нас же во-первых сильное лобби со стороны та...,законы,нарушаются,0,0.077439,0.028327,0.012186,0.002778,0.048373,0.003792,0.016573,0.004931,0.090475,0.315231,0.003851,0.000670
24,Ну что ж может на первых этапах это и будет ра...,этап,имеющий,0,0.052988,0.032011,0.019651,0.040918,0.036316,0.021348,0.113726,0.058758,0.008622,0.346009,0.058232,0.006177
36,Поступай так чтобы твои наибольшие способности...,способности,служили,0,0.081640,0.087470,0.007604,0.054689,0.079828,0.015473,0.110307,0.085634,0.005489,0.265055,0.036467,0.003842
48,Моя работа приносит мне не только моральное уд...,работа,приносит,0,0.078655,0.058814,0.029257,0.083207,0.071137,0.037745,0.076373,0.044260,0.018958,0.449769,0.026439,0.009720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74712,Вообще же несмотря на принятую в России в 2002...,Байконур,остаётся,0,0.041951,0.048825,0.042004,0.053650,0.030706,0.019956,0.061675,0.036742,0.026050,0.318591,0.047989,0.014965
74724,Сегодня это обходится Москве в 115 миллионов в...,это,обходится,0,0.031431,0.062897,0.017560,0.087507,0.151670,0.021914,0.068135,0.030934,0.041089,0.589712,0.189204,0.004188
74736,В этом ключе Байконур говорит учёный не такое ...,Байконур,говорит,0,0.060743,0.093803,0.046511,0.094177,0.049568,0.049816,0.059018,0.070121,0.018455,0.569048,0.088115,0.024967
74748,Впрочем если восстанавливать структуру космодр...,она,была,0,0.004514,0.016066,0.016965,0.019832,0.147516,0.008457,0.101186,0.020928,0.020220,0.850502,0.006349,0.001710


In [ ]:
all_dfs = ['vmean_amean', 'vmean_amax', 'vmean_ast', 'vmax_amean', 'vmax_amax', 'vmax_ast', 'vst_amean', 'vst_amax', 'vst_ast']

In [ ]:
differences = {}
for df in all_dfs:
  differences[df] = check_if_difference_is_ok(globals()[df])

In [ ]:
list(differences)

['vmean_amean',
 'vmean_amax',
 'vmean_ast',
 'vmax_amean',
 'vmax_amax',
 'vmax_ast',
 'vst_amean',
 'vst_amax',
 'vst_ast']

In [ ]:
differences['vmean_amean']

{0: {0: (1, 'Разница: 4347'),
  1: (1, 'Разница: 4431'),
  2: (1, 'Разница: 4775'),
  3: (1, 'Разница: 3837'),
  4: (1, 'Разница: 4955'),
  5: (1, 'Разница: 3861'),
  6: (1, 'Разница: 4897'),
  7: (1, 'Разница: 3767'),
  8: (1, 'Разница: 5173'),
  9: (1, 'Разница: 1701'),
  10: (1, 'Разница: 3965'),
  11: (1, 'Разница: 5189')},
 1: {0: (1, 'Разница: 5551'),
  1: (1, 'Разница: 5085'),
  2: (1, 'Разница: 4915'),
  3: (1, 'Разница: 4845'),
  4: (1, 'Разница: 4595'),
  5: (1, 'Разница: 5321'),
  6: (1, 'Разница: 4333'),
  7: (1, 'Разница: 6021'),
  8: (1, 'Разница: 5013'),
  9: (1, 'Разница: 4173'),
  10: (1, 'Разница: 3911'),
  11: (1, 'Разница: 5439')},
 2: {0: (1, 'Разница: 1893'),
  1: (1, 'Разница: 2439'),
  2: (1, 'Разница: 4919'),
  3: (1, 'Разница: 4789'),
  4: (1, 'Разница: 5039'),
  5: (1, 'Разница: 4661'),
  6: (1, 'Разница: 4151'),
  7: (1, 'Разница: 5645'),
  8: (1, 'Разница: 5141'),
  9: (1, 'Разница: 4953'),
  10: (1, 'Разница: 5129'),
  11: (1, 'Разница: 5299')},
 3: {0: (1